<a href="https://colab.research.google.com/github/chacha715/ESAA_OB/blob/main/OB_%EB%B3%B5%EC%8A%B5_%EB%A9%9C%EB%A1%A0%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[노래 가사 분석을 통한 인기차트 Top10 예측하기](https://rahites.tistory.com/103)

In [35]:
!python --version

import selenium
print(selenium.__version__)

Python 3.10.12
4.11.2


In [36]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 338 kB in 1s (227 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22

In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [38]:
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('—headless')        # Head-less 설정
options.add_argument('—no-sandbox')
options.add_argument('—disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [44]:
driver = webdriver.Chrome(options=options)

**방법**


1. 멜론 플랫폼을 이용하여 1990년부터 2021년까지 연도별로 인기있었던 Top30곡의 노래 가사를 크롤링

2. 노래 가사 데이터를 여러가지 방법으로 텍스트 분석 진행

3. 텍스트 분석을 진행한 결과를 바탕으로 1990 ~ 2019년까지의 가사 데이터를 Train, 2020 ~ 2021년의 가사 데이터를 Test로 분류하여 각각의 Feature data를 생성 ( 이 때 Top10안에 드는 데이터들의 label값을 1로 지정 )

4. 생성한 Feature data를 활용하여 여러 머신러닝 모델을 이용해 Test data의 Top10여부 예측

### 1. 데이터 크롤링

In [45]:
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup

# driver.get("https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate=2021")
driver.get('https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate=2022')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

def melon_collector(url):
    time.sleep(5)
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 제목 가져오기
    title=driver.find_elements(by=By.CLASS_NAME, value='ellipsis.rank01') # 제목

    title2=[]
    for i in title:
        title2.append(i.text)

    del title2[30:]

    # 가수 가져오기
    singer=driver.find_elements(by=By.CLASS_NAME, value='ellipsis.rank02') # 가수

    singer2=[]
    for i in singer:
        singer2.append(i.text)

    del singer2[30:]

    # 가사 가져오기

    song_info = soup.find_all('div', attrs={'class': 'ellipsis rank01'})

    # Top 30만 추출
    songid = []

    for i in range(30):
        try:
            songid.append(re.sub(r'[^0-9]', '', song_info[i].find("a")["href"][43:])) # 곡 id정보 추출
        except:
            songid.append('')
            continue # 1992년 먼지가 되어와 같이 멜론에서 현재 들을 수 없는 노래 제외

    lyrics=[]

    for i in songid:
        try:
            driver.get("https://www.melon.com/song/detail.htm?songId=" + i)
            time.sleep(2)
            lyric=driver.find_element(by=By.CLASS_NAME, value="lyric")
            lyrics.append(lyric.text)
        except: # 가사가 없는경우
            lyrics.append('')
            continue
    lyrics2 = []

    for i in lyrics:
        lyrics2.append(i.replace("\n"," "))

    df=pd.DataFrame({"제목":title2,"가수":singer2,"가사":lyrics2})

    # 저장하기
    df.to_csv(f'멜론{start}.csv', index = False)

In [55]:
# start = 1990
start = 2015
url = 'https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate='

while start<=2022:
    new_url = url + str(start)
    melon_collector(new_url)
    print(start,'완료') # 완료 여부 표시
    start += 1

MaxRetryError: ignored

### 2. 가사 텍스트 분석 & 피처 생성

In [46]:
# Top10인지 아닌지 여부 column 생성
for melon_year in melon_total:
    melon_year['Top10'] = 0


for melon_year in melon_total:
    for i in melon_year.index:
        if i<=10:
            melon_year['Top10'][i] += 1
        else:
            continue

NameError: ignored

In [ ]:
# 가사 column을 따로 지정
gasa_1990s = melon_1990s.가사
gasa_2000s = melon_2000s.가사
gasa_2010s = melon_2010s.가사
gasa_2020s = melon_2020s.가사

# Komoran
from konlpy.tag import Komoran

Komoran = Komoran()

noun2_1990s = []
noun2_2000s = []
noun2_2010s = []
noun2_2020s = []

for i in gasa_1990s:
    try:
        noun2_1990s.append(Komoran.nouns(i))
    except:
        continue

for i in gasa_2000s:
    try:
        noun2_2000s.append(Komoran.nouns(i))
    except:
        continue

for i in gasa_2010s:
    try:
        noun2_2010s.append(Komoran.nouns(i))
    except:
        continue

for i in gasa_2020s:
    try:
        noun2_2020s.append(Komoran.nouns(i))
    except:
        continue

In [ ]:
# 한국어 불용어
with open('stopword.txt', 'r', encoding='utf-8') as f:
    text = f.read()

stopwords = text.split()

# 불용어 처리
noun2_1990s_sw = []; noun2_2000s_sw = []; noun2_2010s_sw=[]; noun2_2020s_sw =[]


# 2
for text in noun2_1990s:
    temp = []
    for word in text:
        if word not in stopwords:
            temp.append(word)
    noun2_1990s_sw.append(temp)

for text in noun2_2000s:
    temp = []
    for word in text:
        if word not in stopwords:
            temp.append(word)
    noun2_2000s_sw.append(temp)

for text in noun2_2010s:
    temp = []
    for word in text:
        if word not in stopwords:
            temp.append(word)
    noun2_2010s_sw.append(temp)

for text in noun2_2020s:
    temp = []
    for word in text:
        if word not in stopwords:
            temp.append(word)
    noun2_2020s_sw.append(temp)

#### 2.1. 빈도 분석

In [ ]:
# Top 30 노래 가사에서 등장한 단어의 빈도
# Komoran
# 가장 많이 등장한 Top10 단어들의 빈도를 Feature로 사용
from collections import Counter

mostwords_1990s = []
mostwords_2000s = []
mostwords_2010s = []
mostwords_2020s = []

for document in noun2_1990s_sw:
    mostwords_1990s.append([y for x,y in Counter(document).most_common(10)])
for document in noun2_2000s_sw:
    mostwords_2000s.append([y for x,y in Counter(document).most_common(10)])
for document in noun2_2010s_sw:
    mostwords_2010s.append([y for x,y in Counter(document).most_common(10)])
for document in noun2_2020s_sw:
    mostwords_2020s.append([y for x,y in Counter(document).most_common(10)])

# DataFrame 형태로 변경
mostwords_1990s = pd.DataFrame(mostwords_1990s).fillna(0)
mostwords_1990s.columns = mostwords_1990s.columns.map(lambda x : '빈도_'+str(x+1)+'등')
mostwords_2000s = pd.DataFrame(mostwords_2000s).fillna(0)
mostwords_2000s.columns = mostwords_2000s.columns.map(lambda x : '빈도_'+str(x+1)+'등')
mostwords_2010s = pd.DataFrame(mostwords_2010s).fillna(0)
mostwords_2010s.columns = mostwords_2010s.columns.map(lambda x : '빈도_'+str(x+1)+'등')
mostwords_2020s = pd.DataFrame(mostwords_2020s).fillna(0)
mostwords_2020s.columns = mostwords_2020s.columns.map(lambda x : '빈도_'+str(x+1)+'등')

# melon dataframe에 가사가 없는 행 제거
melon_1990s.dropna(inplace=True)
melon_2000s.dropna(inplace=True)
melon_2010s.dropna(inplace=True)
melon_2020s.dropna(inplace=True)

In [ ]:
feature = []
feature_te = []

# train
bindo_train = pd.concat([mostwords_1990s, mostwords_2000s, mostwords_2010s], axis=0).reset_index(drop=True)
feature.append(bindo_train)

# test
feature_te.append(mostwords_2020s)

2.2 TF-IDF 분석

In [ ]:
tf_noun_1990s = []
tf_noun_2000s = []
tf_noun_2010s = []
tf_noun_2020s = []

for i in noun2_1990s_sw:
    tf_noun_1990s.append(" ".join(i))
for i in noun2_2000s_sw:
    tf_noun_2000s.append(" ".join(i))
for i in noun2_2010s_sw:
    tf_noun_2010s.append(" ".join(i))
for i in noun2_2020s_sw:
    tf_noun_2020s.append(" ".join(i))

In [ ]:
# 1990년대에 주로 사용된 단어 Top 10으로
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_1990s = TfidfVectorizer(max_features =10)  # 1글자 사용x, L2 정규화
#vectorizer_2000s = TfidfVectorizer()
#vectorizer_2010s = TfidfVectorizer()
#vectorizer_2020s = TfidfVectorizer()

doc_term_mat_1990s_90 = vectorizer_1990s.fit_transform(tf_noun_1990s)
doc_term_mat_d_1990s_90 = doc_term_mat_1990s_90.toarray() # 희소행렬

doc_term_mat_2000s_90 = vectorizer_1990s.transform(tf_noun_2000s)
doc_term_mat_d_2000s_90 = doc_term_mat_2000s_90.toarray()

doc_term_mat_2010s_90 = vectorizer_1990s.transform(tf_noun_2010s)
doc_term_mat_d_2010s_90 = doc_term_mat_2010s_90.toarray()

doc_term_mat_2020s_90 = vectorizer_1990s.transform(tf_noun_2020s)
doc_term_mat_d_2020s_90 = doc_term_mat_2020s_90.toarray()

tfidf_1990s = pd.DataFrame(doc_term_mat_d_1990s_90)
tfidf_2000s = pd.DataFrame(doc_term_mat_d_2000s_90)
tfidf_2010s = pd.DataFrame(doc_term_mat_d_2010s_90)
tfidf_2020s = pd.DataFrame(doc_term_mat_d_2020s_90)
# tfidf_1990s.columns = vectorizer_1990s.get_feature_names_out()
tfidf_1990s.columns = tfidf_1990s.columns.map(lambda x : '1990_' + str(x))
tfidf_2000s.columns = tfidf_2000s.columns.map(lambda x : '1990_' + str(x))
tfidf_2010s.columns = tfidf_2010s.columns.map(lambda x : '1990_' + str(x))
tfidf_2020s.columns = tfidf_2020s.columns.map(lambda x : '1990_' + str(x))

In [ ]:
# train
tfidf_90_train = pd.concat([tfidf_1990s, tfidf_2000s, tfidf_2010s], axis=0).reset_index(drop=True)
feature.append(tfidf_90_train)

# test
feature_te.append(tfidf_2020s)

2.3. 토픽 모델링

In [ ]:
# Perplexity(혼란도) 확률 모델이 결과를 얼마나 정확하게 예측하는지.낮을수록 정확하게 예측
# Coherence Score 을 판단, 토픽이 얼마나 의미론적으로 일관성 있는지, 높을수록 의미론적 일관성 높음

TOPICS_W_NUM =20 # 출력할 토픽별 단어의 개수
save_lda_model=0
RANDOM_STATE = 2020
UPDATE_EVERY = 1
CHUNKSIZE = 100
PASSES = 10
ALPHA = 'auto'
PER_WORD_TOPICS = True
print('NUM_TOPICS', 'perplexity', 'coherence')
for i in range(1,30):
    NUM_TOPICS=i

    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word,
                                              num_topics=NUM_TOPICS, random_state=RANDOM_STATE,
                                              update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                              passes=PASSES, alpha=ALPHA, per_word_topics=PER_WORD_TOPICS)

    doc_lda = lda_model[corpus]

    # Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=result_data, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()

	print('T',NUM_TOPICS, lda_model.log_perplexity(corpus), coherence_lda)

In [ ]:
# pyLDAvis.enable_notebook() # 주피터 노트북 실행시
def create_vis(model):
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim_models.prepare(model, corpus, id2word, sort_topics=False)
    # pyLDAvis.save_html(vis, './Result_lda_vis.html')
    return vis

create_vis(lda_model12) # 12개로 토픽 모델링한 모델